In [1]:
from pyrsistent import s
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

import time
from time import sleep

import json
import requests
import uuid

In [2]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
url = 'https://www.rightmove.co.uk/property-for-sale/find.html?searchType=SALE&locationIdentifier=REGION%5E274&insId=1&radius=10.0&minPrice=&maxPrice=&minBedrooms=&maxBedrooms=&displayPropertyType=&maxDaysSinceAdded=&_includeSSTC=on&sortByPriceDescending=&primaryDisplayPropertyType=&secondaryDisplayPropertyType=&oldDisplayPropertyType=&oldPrimaryDisplayPropertyType=&newHome=&auction=false'

driver.get(url) # look for all properties for sale within a 10-mile radius from Cambridge, UK
time.sleep(2)
        
page = 0
property_number = 0

whole_query_property_links = []
properties_dict_list = []

print('\n---Program initialised.')


---Program initialised.


In [ ]:
def bypass_cookies(self):
    '''A method that bypasses cookies if present.'''
    try:
        self.accept_cookies_button = self.driver.find_element(By.XPATH, value='//button[@class="optanon-allow-all accept-cookies-button"]')
        self.accept_cookies_button.click()
    except:
        pass # passes if there is no cookies button
    time.sleep(2)